# 1. Imports and Definitions

## Library Downloads

In [0]:
!pip install -q keras wandb

## Imports

In [0]:
# generic functions
import numpy as np
import json
import tensorflow as tf
import os

# Keras imports
from keras import (Input as ksInp, Sequential as ksSeq, layers as ksl, optimizers as kso, utils as ksu)
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist # dry runs for code checking

# WandB imports
import wandb
from wandb.keras import WandbCallback as WandbCallback

# 2. Harness Functions

In [0]:
# create config object for WandB sweep (general architecture)
def configForWandbSweep():
  wcfg = wandb.config
  config = { "layers": [{
    "type": "input", 
    "size": wcfg.input_size
  }] }

  #i=1
  while True:
    try:
      config["layers"].append({})
      # need string-based execution for general architecture
      # proceeding with specific architecture suggested for A2
    except KeyError: break

# create config object for WandB sweep (Assignment 2 architecture)
def configForA2WandbSweep():
  wcfg = wandb.config
  return { "layers": [
    {
      "type": "input", 
      "size": wcfg.input_size
    },
    {
      "type": "conv",
      "filter_num": wcfg.layer_1_filter_num,
      "filter_size": wcfg.layer_1_filter_size,
      "activation": wcfg.layer_1_activation
    },
    {
      "type": "pool",
      "size": wcfg.layer_1_pool_size
    },
    {
      "type": "conv",
      "filter_num": wcfg.layer_2_filter_num,
      "filter_size": wcfg.layer_2_filter_size,
      "activation": wcfg.layer_2_activation
    },
    {
      "type": "pool",
      "size": wcfg.layer_2_pool_size
    },
    {
      "type": "conv",
      "filter_num": wcfg.layer_3_filter_num,
      "filter_size": wcfg.layer_3_filter_size,
      "activation": wcfg.layer_3_activation
    },
    {
      "type": "pool",
      "size": wcfg.layer_3_pool_size
    },
    {
      "type": "conv",
      "filter_num": wcfg.layer_4_filter_num,
      "filter_size": wcfg.layer_4_filter_size,
      "activation": wcfg.layer_4_activation
    },
    {
      "type": "pool",
      "size": wcfg.layer_4_pool_size
    },
    {
      "type": "conv",
      "filter_num": wcfg.layer_5_filter_num,
      "filter_size": wcfg.layer_5_filter_size,
      "activation": wcfg.layer_5_activation
    },
    {
      "type": "pool",
      "size": wcfg.layer_5_pool_size
    },
    {
      "type": "flatten"
    },
    {
      "type": "fc",
      "size": wcfg.layer_6_size,
      "activation": wcfg.layer_6_activation
    },
    {
      "type": "fc",
      "size": wcfg.layer_7_size,
      "activation": wcfg.layer_7_activation
    }
    #* add dropout at strategic location
    ],
    "opt": wcfg.optimizer,
    "loss": wcfg.loss,
    "trainparams": {
      "epochs": wcfg.epochs,
      "batch_size": wcfg.btch_size,
      "val_split": wcfg.val_split
    },
    "wandb": 1
  }

# create config object from JSON file
def configFromJSON(arg):
  # input checks
  argtype = type(arg).__name__
  config = None
  if   argtype=="str":
    config = json.load(open(arg,'r')) #* add failsafe
  elif argtype=="TextIOWrapper":
    config = json.load(arg)
  elif argtype=="dict":
    config = arg
  return config

# initialize model from JSON-style config object
def initModel(config):
  # create layer list
  layers = []

  for lconf in config["layers"]:
    ltype = lconf["type"]
    if   ltype=="input":
      layers.append(ksInp( 
        shape = lconf["size"]
      ) )
    
    elif ltype=="conv":
      layers.append( ksl.Conv2D( 
        lconf["filter_num"],
        lconf["filter_size"], 
        activation = lconf["activation"] 
      ) )
    
    elif ltype=="pool":
      layers.append( ksl.MaxPooling2D( 
        pool_size=lconf["size"] 
      ) )
    
    elif ltype=="fc":
      layers.append( ksl.Dense( 
        lconf["size"], 
        activation = lconf["activation"]
      ) )
    
    elif ltype=="flatten":
      layers.append( ksl.Flatten() )
    
    elif ltype=="dropout":
      layers.append( ksl.Dropout(
        lconf["fraction"]
      ) )

  model = ksSeq(layers)

  # compile and return model for given hyperparameters
  exec( "optexec=kso." + config["optimizer"]) #* revise based on final storage pattern used
  opt = locals()["optexec"]
  model.compile( optimizer = opt, loss = config["loss"], metrics = ["accuracy"])
  return model

def trainingRun(config, xdata=xdata, ydata=ydata, train_data=train_data, val_data=val_data, input_mode=1):
  model = initModel(config)
  tp = config["trainparams"]

  # set up WandB callback to train function
  callbacks=[]
  
  try:
    if config["wandb"]:
      callbacks.append(
        WandbCallback(
          monitor="val_loss" #* add generator for dataset-aug. and batch normalization
        )
      )
    else: print("Warning: not using WandB")
  except KeyError: print("Warning: not using WandB")
  
  # run training loop for given number of epochs
  if input_mode == 1:
    model.fit(xdata, ydata, batch_size = tp["batch_size"], epochs = tp["epochs"], validation_split = tp["val_split"], callbacks = callbacks)
  elif input_mode == 2:
    model.fit(train_data, epochs = tp["epochs"], val_data, callbacks = callbacks)


def get_train_val_data(path, config):
  tp = config["trainparams"]

  data_generator = ImageDataGenerator(rescale = 1/255, horizontal_flip = True, validation_split = tp["val_split"])

  train_dataset = data_generator.flow_from_directory(
    path,
    target_size = (800, 800),
    batch_size = tp["batch_size"],
    subset = "training"
  )

  val_dataset = data_generator.flow_from_directory(
    path,
    target_size = (800, 800),
    batch_size = tp["batch_size"],
    subset = "validation"
  )

  return train_dataset, val_dataset

def runWandbSweep():
  run_config = configForA2WandbSweep()
  train_dataset, val_dataset = get_train_val_data(train_path, run_config)
  trainingRun(run_config, train_data=train_dataset, val_data=val_dataset, input_mode=2)
#* add data flattening/reshaping functions
    

# 3. Dataset Initialization and Reshaping

## MNIST digit dataset

In [0]:
# import MNIST dataset
((x_train, y_train), (x_test, y_test)) = mnist.load_data()

# pad with zeros and convert to 32x32 LeNet-5 input
x_train = np.pad(x_train,((0,0),(2,2),(2,2))).reshape((len(x_train),32,32,1))
x_test = np.pad(x_test,((0,0),(2,2),(2,2))).reshape((len(x_test),32,32,1))

# convert to float and normalize
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

# one hot encode target values
y_train = ksu.to_categorical(y_train)
y_test = ksu.to_categorical(y_test)

# 4. Running A Configuration

In [0]:
wandb.init(project="CNN MNIST Test Runs")
run_config = configFromJSON("lenet5.json")
trainingRun(run_config, xdata=x_train, ydata=y_train)

# 5. Sweeps

In [0]:
wandbSweepCfg = {
  "name":"iNaturalist Parameter Sweep", 
  "metric":{
    "name":"val_loss",
    "goal":"minimize"
  }, 
  "method": "bayes", 
  "parameters":{
    # input parameters
    "input_size": { "values" : [[217, 217]] },

    # layer parameters
    "layer_1_filter_num": { "values": [5,10,20] },
    "layer_1_filter_size": { "values": [3,5,7] },
    "layer_1_activation": { "values": ["relu"] },
    "layer_1_pool_size": { "values": [2,3,4] },

    "layer_2_filter_num": { "values": [5,10,20] },
    "layer_2_filter_size": { "values": [3,5,7] },
    "layer_2_activation": { "values": ["relu"] },
    "layer_2_pool_size": { "values": [2,3,4] },

    "layer_3_filter_num": { "values": [5,10,20] },
    "layer_3_filter_size": { "values": [3,5,7] },
    "layer_3_activation": { "values": ["relu"] },
    "layer_3_pool_size": { "values": [2,3,4] },

    "layer_4_filter_num": { "values": [5,10,20] },
    "layer_4_filter_size": { "values": [3,5,7] },
    "layer_4_activation": { "values": ["relu"] },
    "layer_4_pool_size": { "values": [2,3,4] },

    "layer_5_filter_num": { "values": [5,10,20] },
    "layer_5_filter_size": { "values": [3,5,7] },
    "layer_5_activation": { "values": ["relu"] },
    "layer_5_pool_size": { "values": [2,3,4] },

    "layer_6_size": { "values": [256, 512, 1024] },
    "layer_6_activation": { "values": ["tanh"] },

    "layer_7_size": { "values": [10] },
    "layer_7_activation": { "values": ["softmax"] },

    # optimizer and loss
    "optimizer": { "values": ["SGD(lr=0.01, momentum=0.9)"] },
    "loss": { "values": ["categorical_crossentropy"] },

    # training parameters
    "epochs": { "values":[25] },
    "batch_size": { "values":[32, 64] },
    "val_split": { "values": [0.1] },
  }
}

sweepId = wandb.sweep(wandbSweepCfg)
wandb.agent(sweepId, function = runWandbSweep)

# Using inaturalist_12k

In [0]:

train_path = "/content/drive/MyDrive/Sem 8/DL/inaturalist_12K/train"